# CMF — online 2019.
##  Задание по проекту «Индустриальный data-science»

## Предобработка данных

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

Читаем основной датасет

In [2]:
%%time
df = pd.read_hdf('views_sales_stocks.h5')

Wall time: 29.2 s


In [6]:
df.head()

,dt,product_id,product_views,impressions_campaign,impressions_brand,impressions_catalog,checkouts,unit_count,unit_price
0,2017-06-24,2517052217808,8,140,5,541,0,NaN,690.0
1,2017-06-04,2512030288994,0,0,2,0,0,NaN,NaN
2,2017-06-19,8114012262152,0,0,4,0,0,NaN,NaN
3,2017-06-19,2516040221255,0,0,1,3,0,NaN,NaN
4,2017-06-27,2514030408996,1,0,10,0,0,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10909240 entries, 0 to 10909239
Data columns (total 9 columns):
dt                      object
product_id              object
product_views           int64
impressions_campaign    int64
impressions_brand       int64
impressions_catalog     int64
checkouts               int64
unit_count              float64
unit_price              float64
dtypes: float64(2), int64(5), object(2)
memory usage: 832.3+ MB


In [8]:
df.shape

(10909240, 9)

In [9]:
df['product_id'].nunique()

284894

Оптимизирую размер датасета в памяти

In [10]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return None

In [11]:
reduce_mem_usage(df)

Memory usage of dataframe is 832.31 MB
Memory usage after optimization is: 457.77 MB
Decreased by 45.0%


Заполняю NaNs нулями в 'unit_count'

In [12]:
df['unit_count'].fillna(0, inplace=True)

Считывание датасета c описанием товаров

In [13]:
%%time
dfd = pd.read_excel('data_description.xlsx')

Wall time: 24.7 s


In [14]:
dfd.head()

,product_id,group_name,category_name,category,color,seasonality,brand_name
0,2517052217808,Шлепанцы и вьетнамки,Мужская обувь,NaN,Красный,ЛЕТО,LAMALIBOO
1,2512030288994,Туфли,Мужская обувь,No Name,NaN,NaN,NaN
2,8114012262152,Босоножки,Женская обувь,NaN,NaN,NaN,NaN
3,2516040221255,Босоножки,Женская обувь,No Name,Золотой,NaN,NaN
4,2514030408996,Ботильоны,Женская обувь,NaN,Черный,NaN,NaN


In [15]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284894 entries, 0 to 284893
Data columns (total 7 columns):
product_id       284894 non-null object
group_name       284894 non-null object
category_name    284894 non-null object
category         211331 non-null object
color            185870 non-null object
seasonality      87601 non-null object
brand_name       53002 non-null object
dtypes: object(7)
memory usage: 15.2+ MB


In [16]:
dfd.describe()

,product_id,group_name,category_name,category,color,seasonality,brand_name
count,284894,284894,284894,211331,185870,87601,53002
unique,284894,23,2,4,31,5,1694
top,2515061199799,Туфли,Женская обувь,No Name,Черный,ЛЕТО,DERIMOD
freq,1,62828,234286,100061,74055,35086,2065


Удаляю столбцы с большим количеством NaNs

In [21]:
dfd['color'].isna().sum()

99024

In [22]:
dfd['seasonality'].isna().sum()

197293

In [23]:
dfd['brand_name'].isna().sum()

231892

In [24]:
dfd.drop(columns=['color', 'seasonality', 'brand_name'], inplace=True)

Объединяю два датасета

In [25]:
%%time
full_df = df.merge(dfd, on='product_id')

Wall time: 22.5 s


In [26]:
del dfd
del df

Создаю сводный столбец 'impressions'

In [27]:
full_df['impressions'] = full_df['impressions_campaign'] + full_df['impressions_brand'] + full_df['impressions_catalog']

full_df.drop(columns=['impressions_campaign', 'impressions_brand', 'impressions_catalog'],inplace=True)

Обработка категориальных признаков

In [28]:
full_df['category'].fillna('No Name', inplace=True)
full_df['category'] = pd.Categorical(full_df['category'])
full_df['category_code'] = full_df['category'].cat.codes

In [29]:
full_df['group_name'] = pd.Categorical(full_df['group_name'])
full_df['group_name_code'] = full_df['group_name'].cat.codes

In [30]:
full_df['category_name'] = pd.Categorical(full_df['category_name'])
full_df['category_name_code'] = full_df['category_name'].cat.codes

In [31]:
full_df['product_id'] = pd.Categorical(full_df['product_id'])
full_df['product_id_categ'] = full_df['product_id'].cat.codes

Поскольку задача состоит в прогнозировании того, купит ли тот или иной товар посетитель сайта после просмотра рекламы, я удаляю целевой признак и кол-во добавлений в корзину (т.к. это происходит уже после того, как пользователь увидел рекламу и заинтересовался товаром)

In [32]:
#impressions = full_df['impressions']
product_views = full_df['product_views']
y = full_df['unit_count']

In [33]:
full_df.drop(columns=['checkouts', 'product_views', 'unit_count', 'category',
                      'group_name', 'category_name', 'product_id'], inplace=True)

Разбираюсь с пропущенными значениями цены.
Подставляю медианную цену среди соответствующей товарной группы.
Затем оставшиеся пропуски заменяю медианной ценой среди всех цен.

In [34]:
%%time
full_df['unit_price'] = full_df.groupby('group_name_code').transform(lambda x: x.fillna(x.median()))['unit_price']

Wall time: 15.6 s


In [35]:
full_df['unit_price'].fillna(full_df['unit_price'].mean(), inplace=True)

In [36]:
full_df['unit_price'].isna().sum()

0

Разбиваю столбец с датой на категориальные признаки

In [37]:
full_df['dt'] = pd.to_datetime(full_df['dt'])

full_df['year'] = full_df['dt'].dt.year
full_df['month'] = full_df['dt'].dt.month
full_df['week'] = full_df['dt'].dt.week
full_df['day'] = full_df['dt'].dt.day
full_df['weekday'] = full_df['dt'].dt.weekday

week = full_df['week']

date = full_df['dt']
full_df.drop(columns=['dt'], inplace = True)

Данные обработаны. Осталось только снова оптимизировать размер датасета

In [38]:
reduce_mem_usage(full_df)

Memory usage of dataframe is 655.44 MB
Memory usage after optimization is: 301.71 MB
Decreased by 54.0%


## Обучение

При помощи дерева решений попробую предсказать целевой показатель


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Разделяю 70% выборки (X_train, y_train) под обучение и 30% будут отложенной выборкой (X_holdout, y_holdout)

In [40]:
%%time
X_train, X_holdout, y_train, y_holdout = train_test_split(full_df.values, y, test_size=0.3, random_state=17)

Wall time: 5.45 s


In [55]:
%%time
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
tree.fit(X_train, y_train)

Wall time: 59.2 s


Проверяю точность предсказания на отложенной выборке

In [56]:
tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

0.962941812017458

## Результаты
Таким образом, построена предсказательная модель, которая с высокой точностью определяет, будет ли куплен товар по следующим признакам: unit_price, impressions, category_code, group_name_code, category_name_code, product_id_categ, year, month, week, day, weekday


